In [58]:
import requests
import pandas as pd
from datetime import datetime
from math import log, sqrt, exp
from scipy.stats import norm
from scipy.optimize import brentq
import yfinance as yf
import time
from typing import Optional,Literal

In [59]:
# ----------------------------
# 1. Black-Scholes IV Function
# ----------------------------

def black_scholes_price(S, K, T, r, sigma, option_type="call"):
    """
    Calculates the theoretical price of a European option using the Black-Scholes model.

    Args:
        S (float): Current price of the underlying asset.
        K (float): Strike price of the option.
        T (float): Time to expiration in years (e.g., 0.5 for 6 months).
        r (float): Risk-free interest rate (annualized, e.g., 0.05 for 5%).
        sigma (float): Volatility of the underlying asset (annualized).
        option_type (str): Type of the option, either "call" or "put".

    Returns:
        float: The theoretical Black-Scholes option price.
    """
    # Calculate d1 parameter of the Black-Scholes formula.
    # d1 incorporates the asset price, strike price, time, risk-free rate, and volatility.
    d1 = (log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * sqrt(T))
    # Calculate d2 parameter, which is derived from d1.
    d2 = d1 - sigma * sqrt(T)

    # Calculate option price based on option type.
    if option_type == "call":
        # Black-Scholes formula for a call option.
        # norm.cdf(d1) is the cumulative standard normal distribution function of d1.
        return S * norm.cdf(d1) - K * exp(-r * T) * norm.cdf(d2)
    else:
        # Black-Scholes formula for a put option.
        return K * exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

In [60]:
def atm_call_implied_volatility(price, S, T):
    """
    Calculates the implied volatility (IV) of ATM call option using the Black-Scholes model
    and Brent's method to find the root.

    Implied volatility is the volatility that, when input into the Black-Scholes formula,
    yields the current market price of the option.

    Args:
        price (float): The current market price of the option.
        S (float): Current price of the underlying asset.
        T (float): Time to expiration in years.

    Returns:
        float or None: The implied volatility, or None if calculation fails (e.g., no real root found).
    """
    # The risk-free interest rate, expressed as a decimal
    risk_free_rate = 0.04
    try:
        # brentq is a root-finding algorithm. It tries to find a sigma value
        # such that black_scholes_price(S, K, T, r, sigma, option_type) - price = 0.
        # The search range for sigma is typically between a very small positive number (1e-6)
        # and a reasonably large value (5.0, representing 500% volatility).
        return brentq(lambda sigma: black_scholes_price(S, S, T, risk_free_rate, sigma, "call") - price, 1e-6, 5)
    except ValueError:
        # If brentq cannot find a root within the given interval, a ValueError is raised.
        # This can happen if the market price is inconsistent with the Black-Scholes model
        # for any realistic volatility.
        return None

In [ ]:
def get_atm_iv_dataframe(
    ticker_symbol: str
) -> Optional[pd.DataFrame]:
    """
    Retrieves implied volatilities for a specific strike price across all available 
    maturities and returns them in a pandas DataFrame.
    """
    retries = 5
    current_delay = 2
    for i in range(retries):
        try:
            # 1. Create a Ticker object and fetch initial data
            ticker = yf.Ticker(ticker_symbol)
            
            # Get last closing price
            spot_price = ticker.history(period="1d")["Close"][-1]

            # 2. Get all available expiration dates
            expirations = ticker.options
            if not expirations:
                print(f"Warning: No options expiration dates found for {ticker_symbol}.")
                return pd.DataFrame() # Return empty DataFrame

            # 3. Prepare a list to hold records for the DataFrame
            results_list = []
            
            # 4. Loop through each expiration date
            for exp_date in expirations[0]:
                opt_chain = ticker.option_chain(exp_date)
                
                options_df = opt_chain.calls
                matching_option = options_df[options_df['strike'] == spot_price]
                
                if not matching_option.empty:
                    # Extract the implied volatility
                    iv = atm_call_implied_volatility(matching_option.iloc[0]['lastPrice'], spot_price, spot_price)
                    
                    # Create a record for this maturity
                    record = {
                        'Ticker': ticker_symbol.upper(),
                        'SpotPrice': spot_price,
                        'StrikePrice': spot_price,
                        'ExpirationDate': pd.to_datetime(exp_date),
                        'ImpliedVolatility': iv
                    }
                    results_list.append(record)
            
            # 5. Create DataFrame from the list of records
            if not results_list:
                print(f"No Call options found for {ticker_symbol} with a strike of ${spot_price}.")
                return pd.DataFrame()

            result_df = pd.DataFrame(results_list)
            return result_df

        except Exception as e:
            print(f"Attempt {i + 1}/{retries} failed for {ticker_symbol}: {e}")
            if i < retries - 1:
                print(f"Retrying in {current_delay} seconds...")
                time.sleep(current_delay)
                current_delay *= 2  # Exponential backoff
            else:
                print(f"All {retries} attempts failed for {ticker_symbol}. Aborting.")
                return None
    
    return None # Fallback return

In [62]:
if __name__ == '__main__':
    # --- Example 1: Get Call IVs for Apple (AAPL) ---
    ticker = 'AAPL'
    print(f"\n--- Searching for {ticker} ATM CALL option IVs ---")
    
    aapl_iv_df = get_atm_iv_dataframe(ticker)

    if aapl_iv_df is not None:
        if not aapl_iv_df.empty:
            print(f"Successfully retrieved Implied Volatility Term Structure for {ticker}:")
            # Using to_string() ensures all rows are printed without truncation
            print(aapl_iv_df.to_string())
        else:
            print("Search complete, but no options were found matching the criteria.")
    else:
        print("Failed to retrieve data after multiple retries.")

    print("\n" + "="*80 + "\n")

    # --- Example 2: Invalid Ticker ---
    ticker = 'INVALIDTICKERXYZ'
    print(f"\n--- Searching for data for an invalid ticker: {ticker} ---")
    
    invalid_df = get_atm_iv_dataframe(ticker)
    if invalid_df is None:
        print("Function correctly returned None for an invalid ticker after retries.")


--- Searching for AAPL ATM CALL option IVs ---
Attempt 1/5 failed for AAPL: Too Many Requests. Rate limited. Try after a while.
Retrying in 2 seconds...
Attempt 2/5 failed for AAPL: Too Many Requests. Rate limited. Try after a while.
Retrying in 4 seconds...
Attempt 3/5 failed for AAPL: Too Many Requests. Rate limited. Try after a while.
Retrying in 8 seconds...
Attempt 4/5 failed for AAPL: Too Many Requests. Rate limited. Try after a while.
Retrying in 16 seconds...
Attempt 5/5 failed for AAPL: Too Many Requests. Rate limited. Try after a while.
All 5 attempts failed for AAPL. Aborting.
Failed to retrieve data after multiple retries.



--- Searching for data for an invalid ticker: INVALIDTICKERXYZ ---
Attempt 1/5 failed for INVALIDTICKERXYZ: Too Many Requests. Rate limited. Try after a while.
Retrying in 2 seconds...
Attempt 2/5 failed for INVALIDTICKERXYZ: Too Many Requests. Rate limited. Try after a while.
Retrying in 4 seconds...
Attempt 3/5 failed for INVALIDTICKERXYZ: Too Many 

KeyboardInterrupt: 